In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
# import matplotlib.pyplot as plt
from math import *
import time

In [2]:
torch.cuda.set_device(2)
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
# activation function
def activation(x):
    return x * torch.sigmoid(x)

In [4]:
# build ResNet with one blocks
class Net(torch.nn.Module):
    def __init__(self,input_width,layer_width):
        super(Net,self).__init__()
        self.layer_in = torch.nn.Linear(input_width, layer_width)
        self.layer1 = torch.nn.Linear(layer_width, layer_width)
        self.layer2 = torch.nn.Linear(layer_width, layer_width)
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x):
        y = self.layer_in(x)
        y = y + activation(self.layer2(activation(self.layer1(y)))) # residual block 1
        output = self.layer_out(y)
        return output

In [5]:
dimension = 1

In [6]:
input_width,layer_width = dimension, 4

In [7]:
net = Net(input_width,layer_width).cuda() # network for u on gpu

In [8]:
# defination of exact solution
def u_ex(x):     
    temp = 1.0
    for i in range(dimension):
        temp = temp * torch.sin(pi*x[:, i])
    u_temp = 1.0 * temp
    return u_temp.reshape([x.size()[0], 1])

In [9]:
# defination of f(x)
def f(x):
    temp = 1.0
    for i in range(dimension):
        temp = temp * torch.sin(pi*x[:, i])
    u_temp = 1.0 * temp
    f_temp = dimension * pi**2 * u_temp 
    return f_temp.reshape([x.size()[0],1])

In [10]:
# generate points by random
def generate_sample(data_size):
    sample_temp = torch.rand(data_size, dimension)
    return sample_temp.cuda()

In [11]:
def model(x):
    x_temp = x.cuda()
    D_x_0 = torch.prod(x_temp, axis = 1).reshape([x.size()[0], 1]) 
    D_x_1 = torch.prod(1.0 - x_temp, axis = 1).reshape([x.size()[0], 1]) 
    model_u_temp = D_x_0 * D_x_1 * net(x)
    return model_u_temp.reshape([x.size()[0], 1])

In [12]:
# # Xavier normal initialization for weights:
# #             mean = 0 std = gain * sqrt(2 / fan_in + fan_out)
# # zero initialization for biases
# def initialize_weights(self):
#     for m in self.modules():
#         if isinstance(m,nn.Linear):
#             nn.init.xavier_normal_(m.weight.data)
#             if m.bias is not None:
#                 m.bias.data.zero_()

In [13]:
# # Uniform initialization for weights:
# #                  U(a, b)
# #             nn.init.uniform_(tensor, a = 0, b = 1)
# # zero initialization for biases
# def initialize_weights(self):
#     for m in self.modules():
#         if isinstance(m,nn.Linear):
#             nn.init.uniform_(m.weight.data)
#             if m.bias is not None:
#                 m.bias.data.zero_()

In [14]:
# Normal initialization for weights:
#               N(mean = 0, std = 1)
#             nn.init.normal_(tensor, a = 0, b = 1)
# zero initialization for biases
def initialize_weights(self):
    for m in self.modules():
        if isinstance(m,nn.Linear):
            nn.init.normal_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()

In [15]:
initialize_weights(net)

In [16]:
for name,param in net.named_parameters():
                print(name,param.size())
                print(param.detach().cpu())

layer_in.weight torch.Size([4, 1])
tensor([[-1.6188],
        [-0.1575],
        [-0.8171],
        [ 2.1607]])
layer_in.bias torch.Size([4])
tensor([0., 0., 0., 0.])
layer1.weight torch.Size([4, 4])
tensor([[-0.1349, -1.7616,  0.9672,  1.2653],
        [-1.3354,  2.3986, -0.2910,  2.7757],
        [-0.2376, -1.2827, -0.0343,  0.6631],
        [-0.6478, -1.1500,  0.4669,  0.1168]])
layer1.bias torch.Size([4])
tensor([0., 0., 0., 0.])
layer2.weight torch.Size([4, 4])
tensor([[-2.8801, -0.8930,  0.9382, -2.9352],
        [-0.2756, -1.6567, -0.2478, -1.2848],
        [-2.0648, -0.3417,  0.3326, -0.9054],
        [-0.9460,  0.5050,  0.3564, -0.4207]])
layer2.bias torch.Size([4])
tensor([0., 0., 0., 0.])
layer_out.weight torch.Size([1, 4])
tensor([[-0.4350, -0.4412, -0.0079, -0.5451]])
layer_out.bias torch.Size([1])
tensor([0.])


In [17]:
# loss function to DRM by auto differential
def loss_function(x):
#     x = generate_sample(data_size).cuda()
#     x.requires_grad = True
    u_hat = model(x)
    grad_u_hat = torch.autograd.grad(outputs = u_hat, inputs = x, grad_outputs = torch.ones(u_hat.shape).cuda(), create_graph = True)
    grad_u_sq = ((grad_u_hat[0]**2).sum(1)).reshape([len(grad_u_hat[0]), 1])
    part = torch.sum(0.5 * grad_u_sq  - f(x) * u_hat)  / len(x)
    return part

In [18]:
torch.save(net.state_dict(), 'net_params_DRM_ResNet_Normal.pkl')